# The dynamic Malthusian macro model and a few extensions

## 0. <a id='toc0_'></a>[Preamble](#toc0_)

This model analysis project considers a simple dynamic Malthusian model. For the basic model setup we rely on the lecture note of Carl-Johan Dalgaard (2014), which provides a description on the simple dynamic Malthusian model based on Ashraf and Galor (2011). 

**Imports and set magics:**



In [4]:
# importing relevant modules 
import numpy as np
from scipy import optimize
import math
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# import pyfile with model solver
from modelproject import MalthusModelClass

## 1. <a id='toc1_'></a>[Model description](#toc0_)

**Time:** Discrete and indexed by $t\in\{0,1,\dots\}$.

**Production:** Production is described by a Cobb-Douglas production function that assumes that labor i subject to diminishing returns in production.

$$Y_t = (AX)^\alpha L_t^{1-\alpha}, \ \ \ 0<\alpha<1\tag{1}$$

where 
* $t$ denotes time 
* $Y_t$ is output
* $A$ is the constant level of technology
* $L_t$ is labor input
* $X$ is a fixed production factor (land)

For now we ignore that A could be growing: technological change is viewed merely as a series og discrete shocks.

Note that we can then write output per worker as 

$$y_t = (AX/L_t)^\alpha\tag{2}$$

The second key assumption is that the birth rate (i.e. births per capita) rises with per capita income. We can think of $\eta$ as being bounded below one, such that each parent in the ecnonomy uses a fixed fraction, $\eta$ of their income to rear children. Implicitly $\eta$ captures preferences for family size.

$$n_t=\eta y_t, \ \ \ 0<\eta<1\tag{3}$$

We assume no unemployment and the labor force evolves accoring to the transition equation

$$L_{t+1} = n_t L_t + (1-\mu) L_t, \ \ L_0 \ \text{given} \tag{4}$$  

Assuming people work until they die, we can interpret $\mu$ as reflecting mortality.

**Microfoundations of fertility**:

For simplicity assume that all agents receive the average income $y_t$. The budget constraint is then

$$c_t + \delta n_t = y_t(1-\tau) \tag{5}$$ 

where $c_t$ is consumpion an $\lambda$ is the relative price of children.

The utility function is 

$$u_t =\beta \mathrm{log}(c_t) + (1-\beta) \mathrm{log}(n_t) \tag{6}$$

By substitution of the budget constraint, the agents solves the following problem in when choosing between consumption of goods and expenses children.

$$\max_{n} u_t = \beta[y_t (1-\tau) -\delta n_t] + (1-\beta) \mathrm{log} (n_t) \tag{7}$$

## 2. <a id='toc2_'></a>[Solve household problem](#toc0_)

We can start by solving the model numerically for given income, $y=100$:

In [2]:
model = MalthusModelClass() # call class
model.solve() # solving model
print(f"Optimal consumption: {model.c:.2f}")
print(f"Optimal number of off spring: {model.n:.2f}")

Optimal consumption: 49.99
Optimal number of off spring: 16.67


We can also solve the model analytically the household problem analytically

In [3]:
# a. variables
u = sm.symbols('u')
y = sm.symbols('y')
c = sm.symbols('c')
n = sm.symbols('n')

# b. parameters
beta = sm.symbols('beta')
delta = sm.symbols('delta')
tau = sm.symbols('tau')

# c. utility function
u = beta*sm.ln(y*(1-tau)-delta*n)+(1-beta)*sm.log(n)
u


beta*log(-delta*n + y*(1 - tau)) + (1 - beta)*log(n)

In [4]:
# d. differentiate wrt. to n 
du_dn = u.diff(n)
du_dn

-beta*delta/(-delta*n + y*(1 - tau)) + (1 - beta)/n

In [5]:
# e. 
sol = sm.solve(sm.Eq(du_dn,0), n)
sol[0]


y*(beta*tau - beta - tau + 1)/delta

In the following parts we will do derivations using Sympy but state the equilibrium relations.

## 3. <a id='toc2_'></a>[Steady state](#toc0_)

**Population**:

Analytically, the steady state level of $L$ is given by

$$
L^\star = \eta(L^\star)^{1-\alpha}(AX)^\alpha+(1-\mu)L^\star 

\Rightarrow L^\star = \left(\frac{\eta}{\mu}\right)^{1-\alpha}AX
$$

where $L_{t+1}=L_t=L^\star$. 

For the Malthus model we are also interested in the population density

$$L^\star/X$$ 

which rises with greater levels of technology $A$, more ressources for child rearing $\eta$ or if mortality $\mu$ declines. 

In [6]:
# a. call rootfinder using the Brent-Method from model class
print(f'Steady state population is: {model.find_ss_l():.3f}')

# b. calculate population density
print(f'Population density is: {model.find_ss_l()/model.par.X:.3f}')


Steady state population is: 608.220
Population density is: 60.822


**Income**:

By substituting $L_t, L_{t+1}$ and (3) into (4) we get that income per capita evovles according to 

$$ y_{t+1} = \eta^{-\alpha} y_t^{1-\alpha}+(1-\mu)^{-\alpha} y_t $$

And in steady state

$$y_{t+1}=y_t=y^\star=\frac{\mu}{\eta}$$

In [7]:
# a. call rootfinder using the Brent-Method from model class
print(f'Steady state income per capita is: {model.find_ss_y():.3f}')

Steady state income per capita is: 15.000








In finding the steady state above we used **Brent-Method**, which we will shortly sketch here. Specifically from SciPy we use the **brentq** method. It is a root-finding algorithm that combines root bracketing, bisection and inverse quadratic interpolation.

Our goal is to find the zero of the function $f$ on the sign changing interval $[a,b]$. $f(a)$ and $f(b)$ must have opposite signs. 

1. You start from an interval $[a,b]$ that must contain the root. 
2. Evaluate function at the endpoints of the interval. 
3. Based on the sign of the function values, choose either bisection or the secant method to find an approximation of the root. 
4. Update the bracket $[a,b]$
5. If the algorithm reaches a max iter number or fails to converge, raise an error.

When the secand method fails to converge, it switches switch to bisection. The secant method is used when inverse quadratic interpolation does not converge. The inverse quadratic interpolation involves fitting a quadratic curve from the 3 function evaluations above. the quadratic is then used to make a new root approxmimation.

The Brent-Method is considered one of the best root-finding algorithms.

Input: A function f(x) to find the root of, and an initial bracket [a, b] containing the root.

Output: An approximation of the root.

1. Set f(a) and f(b) to the values of the function at the endpoints of the bracket.

2. If f(a) and f(b) have the same sign, then the root is not contained within the bracket and the algorithm terminates.

3. Set c = a and fc = f(a).

4. While the stopping criterion is not met:

a. If f(b) and fc have opposite signs, perform the quadratic interpolation step to find the next guess for the root, x. Otherwise, perform the bisection step to find the midpoint of the bracket, x.

b. If |x-b| >= |b-c|/2, or if |x-b| >= |c-d|/2, then skip the interpolation and perform bisection instead.

c. Evaluate f(x) and update the bracket [a,b] and the points c, d as follows:

i. If f(x) and f(b) have opposite signs, set a = b and b = x.

ii. If f(x) and fc have opposite signs, set d = c and c = x.

iii. If f(x) and fc have the same sign, set the bracket [a,b] to [c,x] or [x,d] depending on which interval contains the root.

d. If the stopping criterion is met (i.e., if |b-a| is smaller than a tolerance level), then return the current estimate of the root, b.

5. If the algorithm reaches the maximum number of iterations without converging, raise an error.

## 4. <a id='toc4_'></a>[Simulating the baseline model](#toc0_)

In simulating the population, we use the derived optimal number of off spring from 2. to insert in (4). We then get

$$L_{t+1} = \left(\frac{1-\beta}{\delta}\right) y_t L_t + (1-\mu) L_t \Leftrightarrow $$  

$$L_{t+1} = \left(\frac{1-\beta}{\delta}\right)(AX)^\alpha L_t^{1-\alpha} + (1-\mu) L_t $$  


In [14]:
model = MalthusModelClass()

In [19]:
model.simulate_malthus_l(100)

TypeError: 'module' object is not callable